Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Llamado de datos a dataframes


In [ ]:
df_maps_restaurantes = pd.read_parquet(r"Generated\Google\metada_sitios.parquet")
df_maps_reviews = pd.read_parquet(r"Generated\Google\merge_site_reviews.parquet")
df_yelp_restaurantes = pd.read_parquet(r"Generated\Yelp\bussines.parquet")
df_yelp_checkin = pd.read_parquet(r"Generated\YELP\business_checkin.parquet")
df_yelp_tips = pd.read_parquet(r"Generated\Yelp\business_tip.parquet")
df_yelp_reviews = pd.read_parquet(r"Generated\Yelp\review.parquet")
df_yelp_users = pd.read_parquet(r"Generated\Yelp\users_extracted.parquet")

### Subway

es la empresa con la que trabajaremos, empezaremos por ver cuantos registros hay en cada Dataframe


In [ ]:
subway_records_rest = df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway"]
count_subway_records_rest = subway_records_rest.shape[0]
print("Número de registros de Subway en restaurantes:", count_subway_records_rest)

In [ ]:
subway_records_checkin = df_yelp_checkin.loc[df_yelp_checkin["name"] == "Subway"]
count_subway_records_checkin = subway_records_checkin.shape[0]
print("Número de registros de Subway en checkin:", count_subway_records_checkin)

In [ ]:
subway_records_tips = df_yelp_tips.loc[df_yelp_tips["name"] == "Subway"]
count_subway_records_tips = subway_records_tips.shape[0]
print("Número de registros de Subway tips:", count_subway_records_tips)

Para revisar el dataframe de reviews y el de usuarios, deberemos obtener el Business_id y usarlo de guía

Para eso, crearemos una lista que contenga todos los business_id que correspondan a locales de Subway, luego crearemos un dataframe que contenga todas las reviews de subway y uno que contenga todos los usuarios que realizaron reviews


In [ ]:
(df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway"]).head(1)

In [ ]:
subway_business_ids = df_yelp_restaurantes.loc[
    df_yelp_restaurantes["name"] == "Subway", "business_id"
].tolist()
len(subway_business_ids)

In [ ]:
reviews_yelp_subway = df_yelp_reviews[
    df_yelp_reviews["business_id"].isin(subway_business_ids)
]
reviews_yelp_subway

Ahora, obtendremos los usuarios

Para hacer esto, obtendremos los user_id de nuestro recien creado **reviews_yelp_subway** y los usaremos como filtro para obtener los usuarios que han cualificado a algun local de subway, ademas, agregaremos una columna que contara cuantas reviews han hecho a subway


In [ ]:
df_yelp_users.head(1)

In [ ]:
unique_user_ids = reviews_yelp_subway["user_id"].unique()
filtered_users = df_yelp_users[df_yelp_users["user_id"].isin(unique_user_ids)]
subway_count = (
    reviews_yelp_subway.groupby("user_id").size().reset_index(name="subway_count")
)
yelp_users_subway = filtered_users.merge(subway_count, on="user_id", how="left")
yelp_users_subway["subway_count"] = yelp_users_subway["subway_count"].fillna(0)
yelp_users_subway.head(1)

# Outlieers


## df_yelp_checkin

Se realiza un boxplot de la puntiación de estrellas y de la cantidad de reviews para visualizar posibles outliers


In [ ]:
df_yelp_checkin.head(1)

In [ ]:
subway_data_checkin = df_yelp_checkin[df_yelp_checkin["name"] == ("Subway")]

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_data_checkin["stars"])
plt.title("Boxplot de Stars para Subway checkin")
plt.ylabel("Stars")

plt.subplot(2, 1, 2)
plt.boxplot(subway_data_checkin["review_count"])
plt.title("Boxplot de Review Count para Subway checkin")
plt.ylabel("Review Count")

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_checkin[
    (df_yelp_checkin["name"] == "Subway") & (df_yelp_checkin["stars"] == 3.5)
]

Vamos a quitar todas aquellas categorias que solo se repiten una vez, pero primero vamos a recorrerlas para tener una noción de lo que tratan


In [ ]:
categorias = df_yelp_checkin["categories"].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {
    categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)
}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:89]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")

Al ser obsoletas todos, prodecemos a su extracción


In [ ]:
categorias = df_yelp_checkin["categories"].str.split(", ").explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[
    frecuencia_categorias == 1
].index.tolist()


def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(", ")
    categorias_filtradas = [
        categoria
        for categoria in categorias_lista
        if categoria not in categorias_solo_una_vez
    ]
    return ", ".join(categorias_filtradas)


df_yelp_checkin["categories"] = df_yelp_checkin["categories"].apply(filtrar_categorias)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_checkin.head()

repetimos el mismo proceso parra **df_yelp_restaurantes**


In [ ]:
subway_rest_data = df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway"]

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_rest_data["stars"])
plt.title("Boxplot de Stars para Subway restaurantes")
plt.ylabel("Stars")

plt.subplot(2, 1, 2)
plt.boxplot(subway_rest_data["review_count"])
plt.title("Boxplot de Review Count para Subway restaurantes")
plt.ylabel("Review Count")

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_restaurantes[
    (df_yelp_restaurantes["name"] == "Subway")
    & (df_yelp_restaurantes["review_count"] > 15)
]

Eliminacion de etiquetas de categoría obsoletos


In [ ]:
categorias = df_yelp_restaurantes["categories"].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {
    categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)
}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:167]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")

In [ ]:
categorias = df_yelp_restaurantes["categories"].str.split(", ").explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[
    frecuencia_categorias == 1
].index.tolist()


def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(", ")
    categorias_filtradas = [
        categoria
        for categoria in categorias_lista
        if categoria not in categorias_solo_una_vez
    ]
    return ", ".join(categorias_filtradas)


df_yelp_restaurantes["categories"] = df_yelp_restaurantes["categories"].apply(
    filtrar_categorias
)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_restaurantes.head()

Repetimos una vez mas el procedimiento pero esta vez con **_df_yelp_tips_**


In [ ]:
subway_data_tips = df_yelp_tips[df_yelp_tips["name"] == "Subway"]

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_data_tips["stars"])
plt.title("Boxplot de Stars para Subway tips")
plt.ylabel("Stars")

plt.subplot(2, 1, 2)
plt.boxplot(subway_data_tips["review_count"])
plt.title("Boxplot de Review Count para Subway tips")
plt.ylabel("Review Count")

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_tips[(df_yelp_tips["name"] == "Subway") & (df_yelp_tips["review_count"] > 20)]

Una vez mas, repetimos el proceso en las categorías


In [ ]:
categorias = df_yelp_tips["categories"].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {
    categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)
}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:44]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")

In [ ]:
categorias = df_yelp_tips["categories"].str.split(", ").explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[
    frecuencia_categorias == 1
].index.tolist()


def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(", ")
    categorias_filtradas = [
        categoria
        for categoria in categorias_lista
        if categoria not in categorias_solo_una_vez
    ]
    return ", ".join(categorias_filtradas)


df_yelp_tips["categories"] = df_yelp_tips["categories"].apply(filtrar_categorias)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_tips.head()

A continuacion se revisará y corregirá en caso de ser necesario el tipo de dato de cada columna


In [ ]:
df_yelp_users.head(1)

In [ ]:
df_yelp_users.info()

In [ ]:
df_yelp_checkin.head(1)

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_restaurantes.head(1)

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_tips.head(1)

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.head(3)

In [ ]:
df_yelp_reviews.info()

Los formatos se encuentran en su mayoria en el estado correcto, se harán los siguientes cambios

- ### df_yelp_checkin, df_yelp_restaurantes df_yelp_tips:
  Se cambia el formato de la columna _is_open_ de Entero a Booleano puesto sus datos eran 1 y 0
- ### df_yelp_users, df_yelp_reviews:
  En las columnas _yelping_since_ (referencia a la fecha de creción del usuario) y _date_ (referencia de la fecha de posteo de la reseña) se les quita la hora puesto que es un dato obsoleto y se cambia su formato a Datetime


In [ ]:
df_yelp_checkin["is_open"] = df_yelp_checkin["is_open"].astype(bool)
df_yelp_restaurantes["is_open"] = df_yelp_restaurantes["is_open"].astype(bool)
df_yelp_tips["is_open"] = df_yelp_tips["is_open"].astype(bool)

In [ ]:
df_yelp_users["yelping_since"] = pd.to_datetime(df_yelp_users["yelping_since"])
df_yelp_reviews["date"] = pd.to_datetime(df_yelp_reviews["date"])
df_yelp_reviews["date"] = df_yelp_reviews["date"].dt.date

In [ ]:
df_maps_restaurantes.info()

In [ ]:
df_maps_reviews.info()

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.info()

In [ ]:
df_yelp_users.info()

# Salida a parquet

In [ ]:
df_maps_restaurantes.to_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews.to_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes.to_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin.to_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips.to_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews.to_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users.to_parquet(r'Generated\Yelp\users_extracted.parquet')

Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# Llamado de datos a dataframes

In [ ]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
# df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
# df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
# df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
# df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
# df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [ ]:
df_maps_restaurantes.info()

In [ ]:
df_maps_reviews.info()

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.info()

In [ ]:
df_yelp_users.info()

# Subway


In [ ]:
# Utiliza str.contains() con una expresión regular para buscar nombres parecidos a "Subway"
df_maps_records_restaurantes = df_maps_restaurantes[df_maps_restaurantes['name'].str.contains(r'Subway', case=False)]

# Calcula el número de registros de Subway en restaurantes
count_subway_records_restaurantes = df_maps_records_restaurantes.shape[0]

# Imprime el número de registros de Subway en restaurantes
print("Número de registros de Subway en restaurantes:", count_subway_records_restaurantes)

In [ ]:
df_maps_records_reviews = df_maps_reviews[df_maps_reviews['gmap_id'].isin(df_maps_records_restaurantes['gmap_id'].tolist())]

# Calcula el número de registros de Subway en reviews
count_subway_records_reviews = df_maps_records_reviews.shape[0]

# Imprime el número de registros de Subway en reviews
print("Número de registros de Subway en reviews:", count_subway_records_reviews)

In [ ]:
df_maps_restaurantes = df_maps_records_restaurantes
df_maps_reviews = df_maps_records_reviews

# Outlieers

## df_maps_restaurantes

In [ ]:
df_maps_restaurantes.info()

### `avg_rating`

Comprobamos que no haya datos fuera del rango (1:5)

In [ ]:
df_maps_restaurantes['avg_rating'].value_counts()

### ``num_of_reviews``

Observamos la distribución de la columna `num_of_reviews`

In [ ]:
# Crear el gráfico de caja
sns.boxplot(x=df_maps_restaurantes['num_of_reviews'], color='blue')

# Añadir etiquetas y título
plt.xlabel('Valores')
plt.title('Distribución num_of_reviews')

# Mostrar el gráfico
plt.show()

Y decidimos eliminar datos que no sean representativos, siendo estos aquellos que posean mas de 160. Recortando asi un 3% `256` de los datos que poseiamos `9489`

In [ ]:
df_maps_restaurantes.drop(df_maps_restaurantes[df_maps_restaurantes['num_of_reviews'] > 160].index, inplace=True) # saco 256 registros de 9489 equivale al 3%

In [ ]:
# Crear el gráfico de caja
sns.boxplot(x=df_maps_restaurantes['num_of_reviews'], color='blue')

# Añadir etiquetas y título
plt.xlabel('Valores')
plt.title('Distribución num_of_reviews')

# Mostrar el gráfico
plt.show()

### ``latitude`` y ``longitude``

Graficamos los datos en un mapa de USA en busca de registros con coordenadas atipicas

In [ ]:
# Cargar los datos geoespaciales del mapa GeoJSON
usa_geojson = 'Geojson/us-states.json'
usa_gpd = gpd.read_file(usa_geojson)

# Crear un GeoDataFrame con los registros de tus datos
registros_gdf = gpd.GeoDataFrame(df_maps_restaurantes, geometry=gpd.points_from_xy(df_maps_restaurantes['longitude'], df_maps_restaurantes['latitude']))

# Crear el subplot
fig, ax = plt.subplots(figsize=(10, 8))

# Plotear los barrios en el mapa
usa_gpd.plot(ax=ax, edgecolor='black', alpha=0.5)

# Plotear los registros en el mapa
registros_gdf.plot(ax=ax, color='red', markersize=1)

# Ajustar diseño
plt.title('Registros en el Mapa')
plt.xlabel('Longitud')
plt.ylabel('Latitud')

# Agregar líneas de división y más valores en el eje horizontal (longitud)
ax.set_xticks(range(-180, -60, 5))  # Establece las marcas en intervalos de 5 grados
ax.set_xticklabels(range(-180, -60, 5), rotation=45)  # Etiquetas rotadas para una mejor legibilidad

# Agregar líneas de división en los ejes x e y
ax.grid(True)

plt.tight_layout()

# Mostrar el mapa
plt.show()

In [ ]:
df_maps_restaurantes.loc[(df_maps_restaurantes['latitude'] < 21) & (df_maps_restaurantes['longitude'] < -70) & (df_maps_restaurantes['longitude'] > -80)]

Buscando en google la direccion me devuelve las correctas coordenadas del local

In [ ]:
df_maps_restaurantes.loc[df_maps_restaurantes['gmap_id'] == "0x8ececf93aaa3553b:0x80d37fdf4ff72429", 'latitude'] = 43.127694
df_maps_restaurantes.loc[df_maps_restaurantes['gmap_id'] == "0x8ececf93aaa3553b:0x80d37fdf4ff72429", 'longitude'] = -89.363469

## Cruze de tablas

Filtramos las reviews dejando solo las que pertenecen a los locales que quedaron luego del tratamiento de outliers

In [ ]:
restaurantes = df_maps_restaurantes['gmap_id'].to_list()
df_maps_reviews = df_maps_reviews[df_maps_reviews['gmap_id'].isin(restaurantes)]
df_maps_reviews.info()

## df_maps_reviews

In [ ]:
df_maps_reviews.info()

### `avg_rating`

Comprobamos que no haya datos fuera del rango (1:5)

In [ ]:
df_maps_reviews['avg_rating'].value_counts()

### ``num_of_reviews``

Observamos la distribución de la columna `num_of_reviews`

In [ ]:
# Crear el gráfico de caja
sns.boxplot(x=df_maps_reviews['num_of_reviews'], color='blue')

# Añadir etiquetas y título
plt.xlabel('Valores')
plt.title('Distribución num_of_reviews')

# Mostrar el gráfico
plt.show()

Y decidimos eliminar datos que no sean representativos, siendo estos aquellos que posean mas de 160. Recortando asi un 3% `256` de los datos que poseiamos `9489`

In [ ]:
df_maps_restaurantes.drop(df_maps_restaurantes[df_maps_restaurantes['num_of_reviews'] > 160].index, inplace=True) # saco 256 registros de 9489 equivale al 3%

In [ ]:
# Crear el gráfico de caja
sns.boxplot(x=df_maps_restaurantes['num_of_reviews'], color='blue')

# Añadir etiquetas y título
plt.xlabel('Valores')
plt.title('Distribución num_of_reviews')

# Mostrar el gráfico
plt.show()

### ``latitude`` y ``longitude``

Graficamos los datos en un mapa de USA en busca de registros con coordenadas atipicas

In [ ]:
# Cargar los datos geoespaciales del mapa GeoJSON
usa_geojson = 'Geojson/us-states.json'
usa_gpd = gpd.read_file(usa_geojson)

# Crear un GeoDataFrame con los registros de tus datos
registros_gdf = gpd.GeoDataFrame(df_maps_reviews, geometry=gpd.points_from_xy(df_maps_reviews['longitude'], df_maps_reviews['latitude']))

# Crear el subplot
fig, ax = plt.subplots(figsize=(10, 8))

# Plotear los barrios en el mapa
usa_gpd.plot(ax=ax, edgecolor='black', alpha=0.5)

# Plotear los registros en el mapa
registros_gdf.plot(ax=ax, color='red', markersize=1)

# Ajustar diseño
plt.title('Registros en el Mapa')
plt.xlabel('Longitud')
plt.ylabel('Latitud')

# Agregar líneas de división y más valores en el eje horizontal (longitud)
ax.set_xticks(range(-180, -60, 5))  # Establece las marcas en intervalos de 5 grados
ax.set_xticklabels(range(-180, -60, 5), rotation=45)  # Etiquetas rotadas para una mejor legibilidad

# Agregar líneas de división en los ejes x e y
ax.grid(True)

plt.tight_layout()

# Mostrar el mapa
plt.show()

# Tipos de datos

## df_maps_restaurantes

In [ ]:
df_maps_restaurantes.info()

Cambiamos el tipo de datos de las columnas `avg_rating`, `num_of_reviews` y `price` para reducir el uso de memoria a cambio de precisiòn innecesaria en los datos

In [ ]:
df_maps_restaurantes['avg_rating'] = df_maps_restaurantes['avg_rating'].astype('float32')

df_maps_restaurantes['num_of_reviews'] = df_maps_restaurantes['num_of_reviews'].astype('int32')

df_maps_restaurantes['price'] = df_maps_restaurantes['price'].astype('category')

## df_maps_reviews

In [ ]:
print(df_maps_reviews.info())

Cambiamos el tipo de datos de las columnas para reducir el uso de memoria.

In [ ]:
# Convertir 'rating' a int8
df_maps_reviews_copy = df_maps_reviews.copy()
df_maps_reviews_copy['rating'] = df_maps_reviews_copy['rating'].astype('int8')

# Convertir 'num_of_reviews' a int32
df_maps_reviews_copy['num_of_reviews'] = df_maps_reviews_copy['num_of_reviews'].astype('int32')

# Convertir 'price' a category
df_maps_reviews_copy['price'] = df_maps_reviews_copy['price'].astype('category')

# Convertir 'anio' a category
df_maps_reviews_copy['anio'] = df_maps_reviews_copy['anio'].astype('category')

# Convertir 'estado' a category
df_maps_reviews_copy['estado'] = df_maps_reviews_copy['estado'].astype('category')

# Convertir 'avg_rating' a float32
df_maps_reviews_copy['avg_rating'] = df_maps_reviews_copy['avg_rating'].astype('float32')

# Convertir 'state_ab' a category
df_maps_reviews_copy['state_ab'] = df_maps_reviews_copy['state_ab'].astype('category')

# Convertir 'us_state' a category
df_maps_reviews_copy['us_state'] = df_maps_reviews_copy['us_state'].astype('category')

# Asignar la copia modificada al DataFrame original
df_maps_reviews = df_maps_reviews_copy

Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
import geopy


# Llamado de datos a dataframes

In [ ]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [ ]:
df_maps_restaurantes.info()

In [ ]:
df_maps_reviews.info()

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.info()

In [ ]:
df_yelp_users.info()

### Subway
es la empresa con la que trabajaremos, empezaremos por ver cuantos registros hay en cada Dataframe


In [ ]:
subway_records_rest = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == 'Subway']
count_subway_records_rest = subway_records_rest.shape[0] 
print("Número de registros de Subway en restaurantes:", count_subway_records_rest)

In [ ]:
subway_records_checkin = df_yelp_checkin.loc[df_yelp_checkin['name'] == 'Subway']
count_subway_records_checkin = subway_records_checkin.shape[0] 
print("Número de registros de Subway en checkin:", count_subway_records_checkin)

In [ ]:
subway_records_tips = df_yelp_tips.loc[df_yelp_tips['name'] == 'Subway']
count_subway_records_tips = subway_records_tips.shape[0] 
print("Número de registros de Subway tips:", count_subway_records_tips)

Para revisar el dataframe de reviews y el de usuarios, deberemos obtener el Business_id y usarlo de guía

Para eso, crearemos una lista que contenga todos los business_id que correspondan a locales de Subway, luego crearemos un dataframe que contenga todas las reviews de subway y uno que contenga todos los usuarios que realizaron reviews

In [ ]:
(df_yelp_restaurantes.loc[df_yelp_restaurantes['name']== 'Subway']).head(1)

In [ ]:
subway_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == 'Subway', 'business_id'].tolist()
len(subway_business_ids)

In [ ]:
reviews_yelp_subway = df_yelp_reviews[df_yelp_reviews['business_id'].isin(subway_business_ids)]
reviews_yelp_subway

Ahora, obtendremos los usuarios

Para hacer esto, obtendremos los user_id de nuestro recien creado **reviews_yelp_subway** y los usaremos como filtro para obtener los usuarios que han cualificado a algun local de subway, ademas, agregaremos una columna que contara cuantas reviews han hecho a subway

In [ ]:
df_yelp_users.head(1)

In [ ]:
unique_user_ids = reviews_yelp_subway['user_id'].unique()
filtered_users = df_yelp_users[df_yelp_users['user_id'].isin(unique_user_ids)]
subway_count = reviews_yelp_subway.groupby('user_id').size().reset_index(name='subway_count')
users_yelp_subway = filtered_users.merge(subway_count, on='user_id', how='left')
users_yelp_subway['subway_count'] = users_yelp_subway['subway_count'].fillna(0)
users_yelp_subway.head(1)

Repetiremos este proceso para obtener los datos de la competencia, serían los siguientes locales:
- Dunkin'
- McDonald's
- Burger King

In [ ]:
# Empezamos con Dunkin
dunkin_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == "Dunkin'", 'business_id'].tolist()
reviews_yelp_dunkin = df_yelp_reviews[df_yelp_reviews['business_id'].isin(dunkin_business_ids)]
unique_user_ids = reviews_yelp_dunkin['user_id'].unique()
filtered_users = df_yelp_users[df_yelp_users['user_id'].isin(unique_user_ids)]
dunkin_count = reviews_yelp_dunkin.groupby('user_id').size().reset_index(name='dunkin_count')
users_yelp_dunkin = filtered_users.merge(dunkin_count, on='user_id', how='left')
users_yelp_dunkin['dunkin_count'] = users_yelp_dunkin['dunkin_count'].fillna(0)
users_yelp_dunkin.head(1)


In [ ]:
# el siguiente es McDonald´s
mc_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == "McDonald's", 'business_id'].tolist()
reviews_yelp_mc = df_yelp_reviews[df_yelp_reviews['business_id'].isin(mc_business_ids)]
unique_user_ids = reviews_yelp_mc['user_id'].unique()
filtered_users = df_yelp_users[df_yelp_users['user_id'].isin(unique_user_ids)]
mc_count = reviews_yelp_mc.groupby('user_id').size().reset_index(name='mc_count')
users_yelp_mc = filtered_users.merge(mc_count, on='user_id', how='left')
users_yelp_mc['mc_count'] = users_yelp_mc['mc_count'].fillna(0)
users_yelp_mc.head(1)


In [ ]:
# terminamos con burguer king
bk_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == "Burger King", 'business_id'].tolist()
reviews_yelp_bk = df_yelp_reviews[df_yelp_reviews['business_id'].isin(bk_business_ids)]
unique_user_ids = reviews_yelp_bk['user_id'].unique()
filtered_users = df_yelp_users[df_yelp_users['user_id'].isin(unique_user_ids)]
bk_count = reviews_yelp_bk.groupby('user_id').size().reset_index(name='bk_count')
users_yelp_bk = filtered_users.merge(bk_count, on='user_id', how='left')
users_yelp_bk['bk_count'] = users_yelp_bk['bk_count'].fillna(0)
users_yelp_bk.head(1)


Una vez hechos los dataframes de tanto el cliente como la competencia, los guardaremos en formato .csv


In [ ]:
users_yelp_subway.to_csv('users_yelp_subway.csv', index=False)
users_yelp_dunkin.to_csv('users_yelp_dunkin.csv', index=False)
users_yelp_mc.to_csv('users_yelp_mc.csv', index=False)
users_yelp_bk.to_csv('users_yelp_bk.csv', index=False)
reviews_yelp_subway.to_csv('reviews_yelp_subway.csv', index=False)
reviews_yelp_dunkin.to_csv('reviews_yelp_dunkin.csv', index=False)
reviews_yelp_mc.to_csv('reviews_yelp_mc.csv', index=False)
reviews_yelp_bk.to_csv('reviews_yelp_bk.csv', index=False)


# Outlieers


## df_yelp_checkin
Se realiza un boxplot de la puntiación de estrellas y de la cantidad de reviews para visualizar posibles outliers

In [ ]:
df_yelp_checkin.head(1)

In [ ]:
subway_data_checkin = df_yelp_checkin[df_yelp_checkin['name']==('Subway')]

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_data_checkin['stars'])
plt.title('Boxplot de Stars para Subway checkin')
plt.ylabel('Stars')

plt.subplot(2, 1, 2)
plt.boxplot(subway_data_checkin['review_count'])
plt.title('Boxplot de Review Count para Subway checkin')
plt.ylabel('Review Count')

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_checkin[(df_yelp_checkin['name'] == 'Subway') & (df_yelp_checkin['stars'] == 3.5)]


Vamos a quitar todas aquellas categorias que solo se repiten una vez, pero primero vamos a recorrerlas para tener una noción de lo que tratan

In [ ]:
categorias = df_yelp_checkin['categories'].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:89]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")


Al ser obsoletas todos, prodecemos a su extracción

In [ ]:
categorias = df_yelp_checkin['categories'].str.split(', ').explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[frecuencia_categorias == 1].index.tolist()
def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(', ')
    categorias_filtradas = [categoria for categoria in categorias_lista if categoria not in categorias_solo_una_vez]
    return ', '.join(categorias_filtradas)
df_yelp_checkin['categories'] = df_yelp_checkin['categories'].apply(filtrar_categorias)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_checkin.head()

repetimos el mismo proceso parra **df_yelp_restaurantes**

In [ ]:
subway_rest_data = df_yelp_restaurantes.loc[df_yelp_restaurantes['name'] == 'Subway']

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_rest_data['stars'])
plt.title('Boxplot de Stars para Subway restaurantes')
plt.ylabel('Stars')

plt.subplot(2, 1, 2)
plt.boxplot(subway_rest_data['review_count'])
plt.title('Boxplot de Review Count para Subway restaurantes')
plt.ylabel('Review Count')

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_restaurantes[(df_yelp_restaurantes['name'] == 'Subway') & (df_yelp_restaurantes['review_count'] > 17)]

Eliminacion de etiquetas de categoría obsoletos

In [ ]:
categorias = df_yelp_restaurantes['categories'].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:167]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")

In [ ]:
categorias = df_yelp_restaurantes['categories'].str.split(', ').explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[frecuencia_categorias == 1].index.tolist()

def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(', ')
    categorias_filtradas = [categoria for categoria in categorias_lista if categoria not in categorias_solo_una_vez]
    return ', '.join(categorias_filtradas)

df_yelp_restaurantes['categories'] = df_yelp_restaurantes['categories'].apply(filtrar_categorias)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_restaurantes.head()


Repetimos una vez mas el procedimiento pero esta vez con ***df_yelp_tips***

In [ ]:
subway_data_tips = df_yelp_tips[df_yelp_tips['name']== 'Subway']

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.boxplot(subway_data_tips['stars'])
plt.title('Boxplot de Stars para Subway tips')
plt.ylabel('Stars')

plt.subplot(2, 1, 2)
plt.boxplot(subway_data_tips['review_count'])
plt.title('Boxplot de Review Count para Subway tips')
plt.ylabel('Review Count')

plt.tight_layout()
plt.show()

In [ ]:
df_yelp_tips[(df_yelp_tips['name'] == 'Subway') & (df_yelp_tips['review_count'] > 20)]


Una vez mas, repetimos el proceso en las categorías 

In [ ]:
categorias = df_yelp_tips['categories'].tolist()
categorias_separadas = [categoria.split(", ") for categoria in categorias]
todas_categorias = [item for sublist in categorias_separadas for item in sublist]
frecuencia_categorias = {categoria: todas_categorias.count(categoria) for categoria in set(todas_categorias)}
categorias_menos_comunes = sorted(frecuencia_categorias.items(), key=lambda x: x[1])
menos_repetidas = categorias_menos_comunes[:44]

print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas:
    print(f"{categoria}: {frecuencia} veces")

In [ ]:
categorias = df_yelp_tips['categories'].str.split(', ').explode()

frecuencia_categorias = categorias.value_counts()

categorias_solo_una_vez = frecuencia_categorias[frecuencia_categorias == 1].index.tolist()

def filtrar_categorias(categoria_str):
    categorias_lista = categoria_str.split(', ')
    categorias_filtradas = [categoria for categoria in categorias_lista if categoria not in categorias_solo_una_vez]
    return ', '.join(categorias_filtradas)
df_yelp_tips['categories'] = df_yelp_tips['categories'].apply(filtrar_categorias)

print("DataFrame modificado eliminando las categorías que se repiten solo una vez:")
df_yelp_tips.head()

A continuacion se revisará y corregirá en caso de ser necesario el tipo de dato de cada columna

In [ ]:
df_yelp_users.head(1)

In [ ]:
df_yelp_users.info()

In [ ]:
df_yelp_checkin.head(1)

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_restaurantes.head(1)

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_tips.head(1)

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.head(3)

In [ ]:
df_yelp_reviews.info()

Los formatos se encuentran en su mayoria en el estado correcto, se harán los siguientes cambios
- ### df_yelp_checkin, df_yelp_restaurantes df_yelp_tips:
 Se cambia el formato de la columna *is_open* de Entero a Booleano puesto sus datos eran 1 y 0
 - ### df_yelp_users, df_yelp_reviews: 
 En las columnas *yelping_since* (referencia a la fecha de creción del usuario) y *date* (referencia de la fecha de posteo de la reseña) se les quita la hora puesto que es un dato obsoleto y se cambia su formato a Datetime

In [ ]:
df_yelp_checkin['is_open'] = df_yelp_checkin['is_open'].astype(bool)
df_yelp_restaurantes['is_open'] = df_yelp_restaurantes['is_open'].astype(bool)
df_yelp_tips['is_open'] = df_yelp_tips['is_open'].astype(bool)

In [ ]:
df_yelp_users['yelping_since'] = pd.to_datetime(df_yelp_users['yelping_since'])
df_yelp_reviews['date'] = pd.to_datetime(df_yelp_reviews['date'])
df_yelp_reviews['date'] = df_yelp_reviews['date'].dt.date

A continuación revisaremos las latitudes y longitudes de los dataframes, con la ayuda de geopandas

En ambos casos (df_yelp_restaurantes y df_yelp_tips) encontramos que hay datos que corresponden a una ciudad de Canadá, datos que serán extraidos


In [ ]:
df_yelp_restaurantes.head(1)

In [ ]:
subway_df_rest = df_yelp_restaurantes[df_yelp_restaurantes['name']== 'Subway']
usa = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
usa.plot(ax=ax, color='lightgrey')
ax.scatter(subway_df_rest['longitude'], subway_df_rest['latitude'], color='red', label='Subway')
plt.title('Ubicación de restaurantes Subway en Estados Unidos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.legend()
plt.show()

In [ ]:
df_yelp_restaurantes = df_yelp_restaurantes.drop(df_yelp_restaurantes[(df_yelp_restaurantes['latitude'] > 50)].index)


In [ ]:
subway_df = df_yelp_restaurantes[df_yelp_restaurantes['name']== 'Subway']
usa = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
usa.plot(ax=ax, color='lightgrey')
ax.scatter(subway_df['longitude'], subway_df['latitude'], color='red', label='Subway')
plt.title('Ubicación de restaurantes Subway en Estados Unidos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.legend()
plt.show()

In [ ]:
subway_df_tips = df_yelp_tips[df_yelp_tips['name']== 'Subway']
usa = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
usa.plot(ax=ax, color='lightgrey')
ax.scatter(subway_df_tips['longitude'], subway_df_tips['latitude'], color='red', label='Subway')
plt.title('Ubicación de restaurantes Subway en Estados Unidos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.legend()
plt.show()

In [ ]:
df_yelp_tips = df_yelp_tips.drop(df_yelp_tips[(df_yelp_tips['latitude'] > 50)].index)

In [ ]:
subway_df_tips = df_yelp_tips[df_yelp_tips['name']== 'Subway']
usa = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
usa.plot(ax=ax, color='lightgrey')
ax.scatter(subway_df_tips['longitude'], subway_df_tips['latitude'], color='red', label='Subway')
plt.title('Ubicación de restaurantes Subway en Estados Unidos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.legend()
plt.show()

Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

# Llamado de datos a dataframes

In [ ]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [ ]:
df_maps_restaurantes.info()

In [ ]:
df_maps_reviews.info()

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_checkin.info()

In [ ]:
df_yelp_tips.info()

In [ ]:
df_yelp_reviews.info()

In [ ]:
df_yelp_users.info()

# Verificacion de valores nulos

Visualizamos los valores nulos de manera grafica con un mapa de calor

In [ ]:
sns.heatmap(df_maps_restaurantes.isnull(), cmap='viridis', cbar=False)
plt.title('Mapa de calor de valores faltantes')
plt.show()

Analizamos el porcentaje de valores nulos presentes en cada columna

In [ ]:
porcnull_rest = df_maps_restaurantes.isnull().mean() * 100
columnasnull_rest = porcnull_rest[porcnull_rest > 0]
columnasnull_rest

Desechamos las columnas irrelevantes con un porcentaje alto de valores nulos 

In [ ]:
df_maps_rest_mod = df_maps_restaurantes.drop(['description','price'],axis=1)

Se rellenan los valores nulos 

In [ ]:
df_maps_rest_mod['hours'] = df_maps_rest_mod['hours'].fillna("").apply(list)
df_maps_rest_mod['state'].fillna(value='N/A',inplace=True)
df_maps_rest_mod['address'].fillna(value='N/A',inplace=True)
df_maps_rest_mod['MISC'] = df_maps_rest_mod['MISC'].fillna("").apply(dict)
df_maps_rest_mod['relative_results'] = df_maps_rest_mod['relative_results'].fillna("").apply(list)

Se verifica la ausencia de valores nulos

In [ ]:
sns.heatmap(df_maps_rest_mod.isnull(), cmap='viridis', cbar=False)
plt.title('Mapa de calor de valores faltantes')
plt.show()

Se realiza el mismo procedimiento

In [ ]:
sns.heatmap(df_maps_reviews.isnull(), cmap='viridis', cbar=False)
plt.title('Mapa de calor de valores faltantes')
plt.show()

In [ ]:
porcnull_review = df_maps_reviews.isnull().mean() * 100
columnasnull_review = porcnull_review[porcnull_review > 0]
columnasnull_review

In [ ]:
df_maps_reviews_mod = df_maps_reviews.drop(['pics','resp'],axis=1)

In [ ]:
df_maps_reviews_mod['text'].fillna(value='N/A',inplace=True)
df_maps_reviews_mod['address'].fillna(value='N/A',inplace=True)
df_maps_reviews_mod['description'].fillna(value='N/A',inplace=True)
df_maps_reviews_mod['price'].fillna(value='-',inplace=True)
df_maps_reviews_mod['hours'] = df_maps_reviews_mod['hours'].fillna("").apply(list)
df_maps_reviews_mod['MISC'] = df_maps_reviews_mod['MISC'].fillna("").apply(dict)
df_maps_reviews_mod['state'].fillna(value='N/A',inplace=True)
df_maps_reviews_mod['relative_results'] = df_maps_reviews_mod['relative_results'].fillna("").apply(list)
df_maps_reviews_mod['state_ab'].fillna(value='N/A',inplace=True)
df_maps_reviews_mod['us_state'].fillna(value='N/A',inplace=True)

In [ ]:
sns.heatmap(df_maps_reviews_mod.isnull(), cmap='viridis', cbar=False)
plt.title('Mapa de calor de valores faltantes')
plt.show()

# Comprobar distribucion de los datos

In [ ]:
df_yelp_reviews['stars'].describe()

La mayoría de las clasificaciones se encuentran en la categoría de 5 estrellas, mientras las clasificaciones de 1 y 2 estrellas son significativamente menores en comparación, lo que sugiere que la mayoría de los usuarios tienden a expresar opiniones muy favorables por encima de las negativas.

In [ ]:
df_yelp_reviews['stars'].value_counts().plot(kind='bar')
plt.title('Distribución de Estrellas')
plt.xlabel('Estrellas')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
df_yelp_reviews['funny'].describe()

Nos indica que casi el 90% de las reseñas carecen de votaciones 'funny'

In [ ]:
df_yelp_reviews['funny'].value_counts().plot(kind='bar')
plt.title('Distribución de votaciones "funny"')
plt.xlabel('Funny')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
df_yelp_reviews['cool'].describe()

Al rededor del 70% de reseñas no cuentan con votaciones 'cool'

In [ ]:
df_yelp_reviews['cool'].value_counts().plot(kind='bar')
plt.title('Distribución de votaciones "cool"')
plt.xlabel('Cool')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
df_yelp_users['review_count'].describe()

In [ ]:
df_yelp_users.boxplot(column='review_count')
plt.title('Boxplot conteo de reseñas')
plt.show()

In [ ]:
df_yelp_users['useful'].describe()

In [ ]:
df_yelp_users.boxplot(column='useful')
plt.title('Boxplot votos "useful"')
plt.show()

In [ ]:
df_yelp_users['funny'].describe()

In [ ]:
df_yelp_users.boxplot(column='funny')
plt.title('Boxplot votos "funny"')
plt.show()

In [ ]:
df_yelp_users['cool'].describe()

In [ ]:
df_yelp_users.boxplot(column='cool')
plt.title('Boxplot votos "cool"')
plt.show()

In [ ]:
df_yelp_users['fans'].describe()

In [ ]:
df_yelp_users.boxplot(column='fans')
plt.title('Boxplot fans')
plt.show()

In [ ]:
df_yelp_users['average_stars'].describe()

In [ ]:
df_yelp_users['average_stars'].hist(bins=10, edgecolor='black')
plt.title('Distribucion de promedio de estrellas')
plt.xlabel('Estrellas')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
df_yelp_users['compliment_hot'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_hot')
plt.title('Boxplot compliment_hot')
plt.show()

In [ ]:
df_yelp_users['compliment_more'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_more')
plt.title('Boxplot compliment_more')
plt.show()

In [ ]:
df_yelp_users['compliment_profile'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_profile')
plt.title('Boxplot compliment_profile')
plt.show()

In [ ]:
df_yelp_users['compliment_cute'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_cute')
plt.title('Boxplot compliment_cute')
plt.show()

In [ ]:
df_yelp_users['compliment_list'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_list')
plt.title('Boxplot compliment_list')
plt.show()

In [ ]:
df_yelp_users['compliment_note'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_note')
plt.title('Boxplot compliment_note')
plt.show()

In [ ]:
df_yelp_users['compliment_plain'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_plain')
plt.title('Boxplot compliment_plain')
plt.show()

In [ ]:
df_yelp_users['compliment_cool'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_cool')
plt.title('Boxplot compliment_cool')
plt.show()

In [ ]:
df_yelp_users['compliment_funny'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_funny')
plt.title('Boxplot compliment_funny')
plt.show()

In [ ]:
df_yelp_users['compliment_writer'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_writer')
plt.title('Boxplot compliment_writer')
plt.show()

In [ ]:
df_yelp_users['compliment_photos'].describe()

In [ ]:
df_yelp_users.boxplot(column='compliment_photos')
plt.title('Boxplot compliment_photos')
plt.show()